## 讀取選定公司的文章去做machine learning & deep learning 方法

In [1]:
import csv
import pandas as pd
import numpy as np
import jieba
import jieba.analyse

In [2]:
df_tgg=pd.read_csv("bda2019_dataset/MTK_stock.csv", encoding="utf-8")
df_tgg=df_tgg.drop(columns=['Unnamed: 0'])
df_tgg.head(5)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,highpriceshift,rate,label
0,2454 聯發科,2018-12-25,221.0,222.0,219.0,221.0,"1,528","337,471","1,229","1,591,506",12.90,1.29,220.0,-0.004525,平
1,2454 聯發科,2018-12-24,223.5,225.5,222.5,225.0,"1,674","375,607","1,173","1,591,506",13.13,1.32,221.0,-0.017778,跌
2,2454 聯發科,2018-12-22,223.0,223.5,221.5,223.0,923,"205,274",860,"1,591,506",13.02,1.31,225.0,0.008969,平
3,2454 聯發科,2018-12-21,224.0,226.0,221.5,223.5,"5,269","1,177,829","2,560","1,591,506",13.05,1.31,223.0,-0.002237,平
4,2454 聯發科,2018-12-20,229.5,231.0,223.5,225.0,"9,042","2,049,881","5,840","1,591,506",13.13,1.32,223.5,-0.006667,平


In [3]:
tnews=pd.read_csv("bda2019_dataset/MTK_content.csv", encoding="utf-8")
tnews=tnews.drop(columns=['Unnamed: 0'])
tnews.head(5)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,label,cutcontent
0,1451783510043_N01,news,yahoo股市,重大要聞,2016-01-03 08:58:00,《各報要聞》30檔電子股，去年可賺1股本,時報資訊,景氣不佳，電子業成了去年放無薪假的大戶，但在一片不景氣聲中，還是有科技公司逆流而上，初估20...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,景氣 不佳 電子業 成 了 去年 放無薪假 的 大戶 但 在 一片 不景氣 聲中 ...
1,1451874408573_N01,news,yahoo股市,重大要聞,2016-01-04 09:59:00,台股開紅盤重摔 CES概念撐腰 IC設計鈺創、凌通、迅杰大漲撐盤,鉅亨網,台股今(4)日2016年第一個交易日，開盤不久指數就面臨賣壓襲擊，摜破8200點與月線支撐，...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,台股 今 ( 4 ) 日 2016 年 第一個 交易日 開盤 不久 指數 就 面臨 賣壓...
2,1451952709026_N01,news,yahoo股市,重大要聞,2016-01-05 07:48:00,《金融》曾銘宗：金融業去年獲利可望再創高,時報資訊,金管會昨（4）日舉辦新春記者會，主委曾銘宗表示，若加計銀行針對中國曝險及不動產的增提準備，2...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,金管會 昨 （ 4 ） 日 舉辦 新春 記者會 主委 曾 銘宗 表示 若 加計 銀行...
3,1451968008054_N01,news,yahoo股市,重大要聞,2016-01-05 12:05:00,台股盤中－TRF風暴 止穩之路遭金融股扯後腿 8100苦戰,鉅亨網,美股昨天重挫但留下200多點下影線，台股今(5)日開低，最低摔到8072點，一度跌破8100...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,美股 昨天 重挫 但 留下 200 多點 下影線 台股 今 ( 5 ) 日 開低 最...
4,1451978807822_N01,news,yahoo股市,重大要聞,2016-01-05 15:07:00,【Y晚報】沙伊交惡！油價只是死貓跳 專家唱衰剩1字頭,Yahoo奇摩股市,美股道瓊昨天連續第三天下挫，收盤大跌276點，不過尾盤拉高收斂跌幅，留下191點的下影線，因...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,美股 道瓊 昨天 連續 第三天 下挫 收盤 大跌 276 點 不過 尾盤 拉高 收斂...


In [4]:
#取前多少個keyword
keywords=list()
with open('bda2019_dataset/companykeyword3000.txt', 'r',encoding='utf-8') as data:
    for keyword in data:
        keywords.append(keyword.strip('\n'))
keywords=keywords[:1500]

# 做TFIDF

In [5]:
#將文章與字典庫做成TFIDF表格
from sklearn.feature_extraction.text import TfidfVectorizer

cv=TfidfVectorizer(vocabulary=keywords) # 預設用只要有空格就一個欄位
# cv=TfidfVectorizer(vocabulary=all0431) # 預設用只要有空格就一個欄位
tfidf=pd.SparseDataFrame(cv.fit_transform(tnews['cutcontent']), 
                       tnews.index,
                       cv.get_feature_names(), 
                       default_fill_value=0)
tfidf=tfidf.drop(columns=['聯發科'])
tfidf.head(7)

,指數,台股,台積電,億元,市場,外資,股價,營收,鴻海,表現,...,股指,超越,顧問處,考慮,之際,吃緊,實質,商品,提高,升值
0,0.000000,0.042315,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048990,...,0.0,0.110506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.032354,0.032933,0.000000,0.000000,0.063210,0.000000,0.230710,0.000000,0.000000,0.152515,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.025018,0.255915,0.027339,0.057616,0.066523,0.000000,0.000000,0.131928,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.066401,0.202772,0.029679,0.033733,0.064865,0.000000,0.039458,0.000000,0.036841,0.039127,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.032650,0.033235,0.000000,0.000000,0.095684,0.033608,0.116412,0.000000,0.000000,0.076956,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.027991,0.024581,0.000000,0.000000,0.000000,0.000000,0.036824,0.030513,0.064813,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.140071,0.040737,0.017887,0.142315,0.019547,0.041195,0.000000,0.026797,0.000000,0.094328,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#把Label放入tfidf表格
tfidf['煞氣的label']=tnews['label']
tfidf.head(7)

,指數,台股,台積電,億元,市場,外資,股價,營收,鴻海,表現,...,超越,顧問處,考慮,之際,吃緊,實質,商品,提高,升值,煞氣的label
0,0.000000,0.042315,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048990,...,0.110506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌
1,0.032354,0.032933,0.000000,0.000000,0.063210,0.000000,0.230710,0.000000,0.000000,0.152515,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌
2,0.000000,0.000000,0.025018,0.255915,0.027339,0.057616,0.066523,0.000000,0.000000,0.131928,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌
3,0.066401,0.202772,0.029679,0.033733,0.064865,0.000000,0.039458,0.000000,0.036841,0.039127,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌
4,0.032650,0.033235,0.000000,0.000000,0.095684,0.033608,0.116412,0.000000,0.000000,0.076956,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌
5,0.000000,0.027991,0.024581,0.000000,0.000000,0.000000,0.000000,0.036824,0.030513,0.064813,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌
6,0.140071,0.040737,0.017887,0.142315,0.019547,0.041195,0.000000,0.026797,0.000000,0.094328,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌


In [7]:
tfidf2=tfidf.drop(columns=['煞氣的label'])

In [8]:
#把資料級切成train跟test
from sklearn import model_selection
X = tfidf2.to_numpy()
Y = tfidf['煞氣的label'].to_numpy()
validation_size = 0.20
seed = 20
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, 
                                                                                random_state=seed, stratify=Y)

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [10]:
#SVM
from sklearn.svm import SVC

SVC_model = SVC(kernel='linear')
SVC_model.fit(X_train,Y_train)
predictions = SVC_model.predict(X_validation)

print(SVC_model.score(X_train,Y_train))
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.625365141187926
0.4459591041869523
[[316  67  45]
 [185  84  50]
 [173  49  58]]
              precision    recall  f1-score   support

           平       0.47      0.74      0.57       428
           漲       0.42      0.26      0.32       319
           跌       0.38      0.21      0.27       280

   micro avg       0.45      0.45      0.45      1027
   macro avg       0.42      0.40      0.39      1027
weighted avg       0.43      0.45      0.41      1027



In [11]:
#SVM K-fold
SVC_model_1 = SVC(kernel='linear')
kfold=model_selection.RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=seed)

result=model_selection.cross_val_score(SVC_model_1, X, Y, cv=kfold, scoring='accuracy')
print(result)
print('%f' % result.mean() )

[0.43579767 0.40758755 0.43190661 0.40253411 0.45170732]
0.425907


In [12]:
#Decision tree
from sklearn.tree import DecisionTreeClassifier

decision_model = DecisionTreeClassifier()
decision_model.fit(X_train,Y_train)
predictions1 = decision_model.predict(X_validation)

print(decision_model.score(X_train, Y_train))
print(accuracy_score(Y_validation, predictions1))
print(confusion_matrix(Y_validation, predictions1))
print(classification_report(Y_validation, predictions1))

0.9936708860759493
0.41674780915287246
[[209 123  96]
 [125 117  77]
 [105  73 102]]
              precision    recall  f1-score   support

           平       0.48      0.49      0.48       428
           漲       0.37      0.37      0.37       319
           跌       0.37      0.36      0.37       280

   micro avg       0.42      0.42      0.42      1027
   macro avg       0.41      0.41      0.41      1027
weighted avg       0.42      0.42      0.42      1027



In [13]:
#Decision tree K-fold
decision_model_1 = DecisionTreeClassifier()

result=model_selection.cross_val_score(decision_model_1, X, Y, cv=kfold, scoring='accuracy')
print(result)
print('%f' % result.mean() )

[0.38521401 0.3959144  0.40272374 0.38499025 0.40097561]
0.393964


In [14]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

forest_model=RandomForestClassifier()
forest_model.fit(X_train,Y_train)
predictions2=forest_model.predict(X_validation)

print(forest_model.score(X_train,Y_train))
print(accuracy_score(Y_validation, predictions2))
print(confusion_matrix(Y_validation, predictions2))
print(classification_report(Y_validation, predictions2))

/home/daniel/anaconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9802823758519961
0.433300876338851
[[274  91  63]
 [164 105  50]
 [145  69  66]]
              precision    recall  f1-score   support

           平       0.47      0.64      0.54       428
           漲       0.40      0.33      0.36       319
           跌       0.37      0.24      0.29       280

   micro avg       0.43      0.43      0.43      1027
   macro avg       0.41      0.40      0.40      1027
weighted avg       0.42      0.43      0.42      1027



In [15]:
#Random forest K-fold
forest_model_1 = DecisionTreeClassifier()

result=model_selection.cross_val_score(forest_model_1, X, Y, cv=kfold, scoring='accuracy')
print(result)
print('%f' % result.mean() )

[0.38035019 0.41439689 0.39007782 0.37621832 0.4       ]
0.392209


In [16]:
#MLP
from sklearn.neural_network import MLPClassifier

model_MLP = MLPClassifier(hidden_layer_sizes=(200,))
model_MLP.fit(X_train, Y_train)

predictions = model_MLP.predict(X_validation)
print(model_MLP.score(X_train,Y_train))
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.9931840311587147
0.4459591041869523
[[219 127  82]
 [107 128  84]
 [102  67 111]]
              precision    recall  f1-score   support

           平       0.51      0.51      0.51       428
           漲       0.40      0.40      0.40       319
           跌       0.40      0.40      0.40       280

   micro avg       0.45      0.45      0.45      1027
   macro avg       0.44      0.44      0.44      1027
weighted avg       0.45      0.45      0.45      1027



/home/daniel/anaconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [17]:
#MLP K-fold
model_MLP_1 = DecisionTreeClassifier()

result=model_selection.cross_val_score(model_MLP_1, X, Y, cv=kfold, scoring='accuracy')
print(result)
print('%f' % result.mean() )

[0.38910506 0.37937743 0.41536965 0.38011696 0.3902439 ]
0.390843


In [18]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
## 设置字体
# from matplotlib.font_manager import FontProperties
# fonts = FontProperties(fname = "/Library/Fonts/华文细黑.ttf",size=14)
%config InlineBackend.figure_format = 'retina'
# %matplotlib inline
import re

Using TensorFlow backend.


In [19]:
stopwords=list()
with open('bda2019_dataset/stopword.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))

In [20]:
stnews=tnews
stnews=stnews.reset_index(drop=True)
stnews.head(3)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,label,cutcontent
0,1451783510043_N01,news,yahoo股市,重大要聞,2016-01-03 08:58:00,《各報要聞》30檔電子股，去年可賺1股本,時報資訊,景氣不佳，電子業成了去年放無薪假的大戶，但在一片不景氣聲中，還是有科技公司逆流而上，初估20...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,景氣 不佳 電子業 成 了 去年 放無薪假 的 大戶 但 在 一片 不景氣 聲中 ...
1,1451874408573_N01,news,yahoo股市,重大要聞,2016-01-04 09:59:00,台股開紅盤重摔 CES概念撐腰 IC設計鈺創、凌通、迅杰大漲撐盤,鉅亨網,台股今(4)日2016年第一個交易日，開盤不久指數就面臨賣壓襲擊，摜破8200點與月線支撐，...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,台股 今 ( 4 ) 日 2016 年 第一個 交易日 開盤 不久 指數 就 面臨 賣壓...
2,1451952709026_N01,news,yahoo股市,重大要聞,2016-01-05 07:48:00,《金融》曾銘宗：金融業去年獲利可望再創高,時報資訊,金管會昨（4）日舉辦新春記者會，主委曾銘宗表示，若加計銀行針對中國曝險及不動產的增提準備，2...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,金管會 昨 （ 4 ） 日 舉辦 新春 記者會 主委 曾 銘宗 表示 若 加計 銀行...


In [21]:
#把label變成數字 keras的Label不能讀中文
stnews.loc[stnews['label']=='漲', 'label'] = 2
stnews.loc[stnews['label']=='跌', 'label'] = 1
stnews.loc[stnews['label']=='平', 'label'] = 0
stnews.head(10)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,label,cutcontent
0,1451783510043_N01,news,yahoo股市,重大要聞,2016-01-03 08:58:00,《各報要聞》30檔電子股，去年可賺1股本,時報資訊,景氣不佳，電子業成了去年放無薪假的大戶，但在一片不景氣聲中，還是有科技公司逆流而上，初估20...,https://tw.stock.yahoo.com/news_content/url/d/...,1,景氣 不佳 電子業 成 了 去年 放無薪假 的 大戶 但 在 一片 不景氣 聲中 ...
1,1451874408573_N01,news,yahoo股市,重大要聞,2016-01-04 09:59:00,台股開紅盤重摔 CES概念撐腰 IC設計鈺創、凌通、迅杰大漲撐盤,鉅亨網,台股今(4)日2016年第一個交易日，開盤不久指數就面臨賣壓襲擊，摜破8200點與月線支撐，...,https://tw.stock.yahoo.com/news_content/url/d/...,1,台股 今 ( 4 ) 日 2016 年 第一個 交易日 開盤 不久 指數 就 面臨 賣壓...
2,1451952709026_N01,news,yahoo股市,重大要聞,2016-01-05 07:48:00,《金融》曾銘宗：金融業去年獲利可望再創高,時報資訊,金管會昨（4）日舉辦新春記者會，主委曾銘宗表示，若加計銀行針對中國曝險及不動產的增提準備，2...,https://tw.stock.yahoo.com/news_content/url/d/...,1,金管會 昨 （ 4 ） 日 舉辦 新春 記者會 主委 曾 銘宗 表示 若 加計 銀行...
3,1451968008054_N01,news,yahoo股市,重大要聞,2016-01-05 12:05:00,台股盤中－TRF風暴 止穩之路遭金融股扯後腿 8100苦戰,鉅亨網,美股昨天重挫但留下200多點下影線，台股今(5)日開低，最低摔到8072點，一度跌破8100...,https://tw.stock.yahoo.com/news_content/url/d/...,1,美股 昨天 重挫 但 留下 200 多點 下影線 台股 今 ( 5 ) 日 開低 最...
4,1451978807822_N01,news,yahoo股市,重大要聞,2016-01-05 15:07:00,【Y晚報】沙伊交惡！油價只是死貓跳 專家唱衰剩1字頭,Yahoo奇摩股市,美股道瓊昨天連續第三天下挫，收盤大跌276點，不過尾盤拉高收斂跌幅，留下191點的下影線，因...,https://tw.stock.yahoo.com/news_content/url/d/...,1,美股 道瓊 昨天 連續 第三天 下挫 收盤 大跌 276 點 不過 尾盤 拉高 收斂...
5,1452039437423_N01,news,yahoo股市,重大要聞,2016-01-06 08:04:00,《外資》台股電子股，麥格理喊進6檔,時報資訊,麥格理資本證券昨（5）日發表2016年大中華科技產業研究報告，認為產品轉型與新產品週期將重塑...,https://tw.stock.yahoo.com/news_content/url/d/...,1,麥格理 資本 證券 昨 （ 5 ） 日 發表 2016 年 大中華 科技 產業 研究 報告 ...
6,1452070032002_N01,news,yahoo股市,重大要聞,2016-01-06 16:17:00,日盛投顧：台股極短線指數區間7900-8250,Moneydj理財網,1.週三台股受到法人賣超權值股影響，指數開低回測8000，雖在陸股等亞股上漲影響之下，盤中指...,https://tw.stock.yahoo.com/news_content/url/d/...,1,1 . 週三 台股 受到 法人 賣超 權值股 影響 指數 開低 回測 8000 雖 ...
7,1452126731305_N01,news,yahoo股市,重大要聞,2016-01-07 07:58:00,工商時報晨訊,財訊快報,◆新台幣連八貶，創80個月新低，昨收33.453元，農曆封關前可能貶破34元。<BR> ◆人...,https://tw.stock.yahoo.com/news_content/url/d/...,2,◆ 新台幣 連八貶 創 80 個 月 新低 昨收 33.453 元 農曆 封關 ...
8,1452126732568_N01,news,yahoo股市,重大要聞,2016-01-07 07:58:00,經濟日報晨訊,財訊快報,◆陸大降官價，人民幣重貶。<BR> ◆亞幣競貶，台幣跌向33.5元，昨收33.453元，逾六...,https://tw.stock.yahoo.com/news_content/url/d/...,2,◆ 陸大 降官 價 人民幣 重貶 。 < BR > ◆ 亞幣競 貶 台幣 跌向 ...
9,1452135732556_N01,news,yahoo股市,重大要聞,2016-01-07 10:35:00,《台北股市》陸股跳水，台股暴跌百點破10年線,時報資訊,北韓核爆成功、國際原油價格跌跌不休與蘋果手機i6s首季大減產，除引爆歐美主要股市在周三大跌外...,https://tw.stock.yahoo.com/news_content/url/d/...,2,北韓 核爆 成功 、 國際 原油 價格 跌跌 不休 與 蘋果 手機 i6s 首季 大 減產 ...


In [22]:
# #把文章重切(扣除掉stopword)放入cutcontent
# corpus = list()
# for i in range(len(stnews)):
#     content=stnews.loc[i,'content']
#     content=re.sub('[a-zA-Z0-9\W]', '', content)
#     seglist=jieba.cut(content.replace('\n','').replace('，',' '),cut_all=False)
#     seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist))
#     cutcontent=' '.join(seglist)
#     corpus.append(cutcontent)
# stnews["cutcontent"]=corpus
# stnews.head(3)

In [23]:
#切成train test 資料
from sklearn import model_selection
X = stnews['cutcontent'].to_numpy()
Y = stnews['label'].to_numpy()
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, 
                                                                                random_state=seed, stratify=Y)

In [24]:
#對label編碼
le = LabelEncoder()
Y_train = le.fit_transform(Y_train).reshape(-1,1)
Y_validation = le.transform(Y_validation).reshape(-1,1)

In [25]:
Y_validation

array([[0],
       [2],
       [2],
       ...,
       [0],
       [1],
       [2]])

In [26]:
#對label做one-hot encoding
ohe = OneHotEncoder()
Y_train = ohe.fit_transform(Y_train).toarray()
Y_validation = ohe.transform(Y_validation).toarray()

/home/daniel/anaconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [27]:
Y_validation

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [28]:
# 使用Tokenizer對詞組進行編碼
## 当我们创建了一个Tokenizer对象后，使用该对象的fit_on_texts()函数，以空格去识别每个词,
## 可以将输入的文本中的每个词编号，编号是根据词频的，词频越大，编号越小。
max_words = 50000
max_len = 1000
tok = Tokenizer(num_words=max_words)  ## 使用的最大词语数为5000
tok.fit_on_texts(stnews['cutcontent'])

In [29]:
## 对每个词编码之后，每句新闻中的每个词就可以用对应的编码表示，即每条新闻可以转变成一个向量了：
train_seq = tok.texts_to_sequences(X_train)
val_seq = tok.texts_to_sequences(X_validation)
## 将每个序列调整为相同的长度
train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
val_seq_mat = sequence.pad_sequences(val_seq,maxlen=max_len)

print(train_seq_mat.shape)
print(val_seq_mat.shape)

(4108, 1000)
(1027, 1000)


In [30]:
#建立LSTM模型

model = Sequential()
model.add(Embedding(max_words+1, 256, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu',name='d1'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu',name='d2'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 256)         12800256  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1000, 256)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
d1 (Dense)                   (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
d2 (Dense)           

In [ ]:
model_fit = model.fit(train_seq_mat,Y_train,batch_size=128,epochs=10,
                      validation_data=(val_seq_mat,Y_validation))


Instructions for updating:
Use tf.cast instead.
Train on 4108 samples, validate on 1027 samples
Epoch 1/10
4108/4108 [==============================] - 33s 8ms/step - loss: 1.0893 - acc: 0.4094 - val_loss: 1.0831 - val_acc: 0.4167
Epoch 2/10
4108/4108 [==============================] - 28s 7ms/step - loss: 1.0775 - acc: 0.4175 - val_loss: 1.0841 - val_acc: 0.4167
Epoch 3/10
4108/4108 [==============================] - 29s 7ms/step - loss: 1.0420 - acc: 0.4645 - val_loss: 1.1116 - val_acc: 0.3973
Epoch 4/10
4108/4108 [==============================] - 28s 7ms/step - loss: 0.8781 - acc: 0.5650 - val_loss: 1.2080 - val_acc: 0.3934
Epoch 5/10
4108/4108 [==============================] - 28s 7ms/step - loss: 0.6971 - acc: 0.6670 - val_loss: 1.5815 - val_acc: 0.3622
Epoch 6/10
4108/4108 [==============================] - 28s 7ms/step - loss: 0.5544 - acc: 0.7517 - val_loss: 1.6947 - val_acc: 0.3759
Epoch 7/10
1920/4108 [=============>................] - ETA: 13s - loss: 0.4102 - acc: 0.8370